In [1]:
!pip install llama_index
!pip install qdrant_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully unins

In [2]:
import tqdm
import os
os.environ["OPENAI_API_KEY"]="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [5]:
!mkdir data
#!wget "https://github.com/run-llama/llama-datasets/blob/main/llama_datasets/10k/uber_2021/source_files/uber_2021.pdf" -o "data/uber_2021.pdf"

mkdir: cannot create directory ‘data’: File exists


In [6]:
from pathlib import Path
from llama_index import download_loader
PDFReader=download_loader("PDFReader")
loader=PDFReader()
documents=loader.load_data(file=Path("./data/uber_2021.pdf"))

In [7]:
len(documents)

160

In [8]:
print(documents[0].text)
print(documents[1].text)

Uber 
2021 
Annual Report
Uber’s Mission
We reimagine the way the world moves for the better
We are Uber. The go-getters. The kind of people who are relentless about our mission to help people go anywhere and get anything  and earn their way . 
Movement is what we  power . It’s our lifeblood. It runs through our veins. It’s what gets us out of bed each morning. It pushes us to constantly reimagine 
how we can move better. For you. For all the places you want to go. For all the things you want to get. For all the ways you want to earn. Across the entire 
world. In real time . At the incredible speed of now.
Our Values
Our values reflect who we are and where we’re going. They guide our decision-making, unite and define our culture, and tell a story to the world about 
Uber’s corporate purpose.
Do the right thingPeriod.
Go get itBring the mindset of a champion.
Our ambition is what drives us to achieve our mission. How we define a champion mindset 
isn't based on how we perform on our bes

In [9]:
print(documents[0])
print(documents[1])

Doc ID: e1808360-b9cc-430b-9f5f-cf52ec5e8c62
Text: Uber  2021  Annual Report
Doc ID: da8c62c7-7396-4c08-99bd-062d4e57b802
Text: Uber’s Mission We reimagine the way the world moves for the
better We are Uber. The go-getters. The kind of people who are
relentless about our mission to help people go anywhere and get
anything  and earn their way .  Movement is what we  power . It’s our
lifeblood. It runs through our veins. It’s what gets us out of bed
each morning. It pushes...


In [10]:
from llama_index.llms import OpenAI

In [11]:
llm=OpenAI(model="gpt-3.5-turbo",temperature=0.1)

In [12]:
from llama_index.embeddings import HuggingFaceEmbedding
embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [13]:
from llama_index import ServiceContext
service_context=ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model

)

In [14]:
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore
client=qdrant_client.QdrantClient(path="./qdrant_db")
vector_store= QdrantVectorStore(client=client,collection_name="rag_customization")

In [16]:
from llama_index.storage.storage_context import StorageContext

In [18]:
storage_context=StorageContext.from_defaults(vector_store=vector_store)

In [19]:
from llama_index import VectorStoreIndex
index=VectorStoreIndex.from_documents(
    documents[:100],
    storage_context=storage_context,
    service_context=service_context,
    show_progress=True
)

Parsing nodes:   0%|          | 0/100 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/569 [00:00<?, ?it/s]

In [20]:
query_engine=index.as_query_engine(similarity_top_k=5)

In [23]:
response=query_engine.query(
    "what is the main objective of this document"
)
print(response)

The main objective of this document is not clear based on the provided context information.


In [24]:
response=query_engine.query(
    "what is the ubers mission"
)
print(response)

Uber's mission is to reimagine the way the world moves for the better. They are relentless about their mission to help people go anywhere, get anything, and earn their way. They strive to constantly reimagine how they can move better for their customers across the entire world in real time.


In [25]:
response=query_engine.query(
    "what is the ubers main business"
)
print(response)

Uber's main business is to provide transportation services and enable people to go anywhere and get anything they need.


In [28]:
response=query_engine.query(
    "what is the delivery taliking about under ITEM 1. Business"
)
print(response)

The delivery is discussing different subjects related to ITEM 1. Business.


In [30]:
response=query_engine.query(
    "platform synergies"
)
print(response)

Uber's values and mission statement do not directly mention "platform synergies."


In [38]:
response=query_engine.query(
    "in this document Research and Development what is main information"
)
print(response)

Empty Response


In [39]:
from llama_index.indices import SimpleKeywordTableIndex
keyword_table_index=SimpleKeywordTableIndex.from_documents(
    documents[:100],
    service_context=service_context,
    show_progress=True

)

Parsing nodes:   0%|          | 0/100 [00:00<?, ?it/s]

Extracting keywords from nodes:   0%|          | 0/569 [00:00<?, ?it/s]

In [40]:
keyword_table_retriever=keyword_table_index.as_retriever()

In [41]:
query_engine=keyword_table_index.as_query_engine(retriever=keyword_table_retriever)

In [42]:
response=query_engine.query(
    "in this document Research and Development what is main information"
)
print(response)

Empty Response


In [43]:
query_engine=keyword_table_index.as_query_engine()
response=query_engine.query(
    "in this document Research and Development what is main information"
)
print(response)

Empty Response
